In [6]:
import json
import pandas as pd
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
from typing import Any, Dict, List
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import xml.etree.ElementTree as ET

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
ls /content/drive/MyDrive/BIT3474-Group5/

ls: cannot access '/content/drive/MyDrive/BIT3474-Group5/': No such file or directory


# **Data Loading**

Imports NVD data from 2022-2024 and creates a compiled dataframe with all the cpe data and all the cve data from those years.

In [7]:
# Helper code used to import NVD data from 2022.
def process_nvd_json(file_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Read the JSON file
    with open(file_path, 'r', encoding = 'ISO-8859-1') as f:
        nvd_data = json.load(f)

    # Lists to store processed items
    cve_items = []
    cpe_items = []

    for cve_item in nvd_data['CVE_Items']:
        cve_data = {}

        # Basic CVE information
        cve_id = cve_item['cve']['CVE_data_meta']['ID']
        cve_data['id'] = cve_id
        cve_data['assigner'] = cve_item['cve']['CVE_data_meta']['ASSIGNER']
        cve_data['published_date'] = cve_item['publishedDate']
        cve_data['last_modified_date'] = cve_item['lastModifiedDate']

        # Description
        descriptions = cve_item['cve']['description']['description_data']
        cve_data['description'] = next((desc['value'] for desc in descriptions if desc['lang'] == 'en'), '')

        # Problem type (CWE)
        try:
            problemtype_data = cve_item['cve']['problemtype']['problemtype_data']
            if problemtype_data and problemtype_data[0]['description']:
                cve_data['cwe'] = problemtype_data[0]['description'][0].get('value', '')
            else:
                cve_data['cwe'] = ''
        except (KeyError, IndexError):
            cve_data['cwe'] = ''

        # References
        try:
            references = cve_item['cve']['references']['reference_data']
            cve_data['references'] = '; '.join(ref['url'] for ref in references)
        except (KeyError, IndexError):
            cve_data['references'] = ''

        # CVSS v3 metrics
        try:
            cvss3 = cve_item['impact']['baseMetricV3']['cvssV3']
            cve_data['cvss3_vector'] = cvss3.get('vectorString', '')
            cve_data['cvss3_base_score'] = cvss3.get('baseScore', None)
            cve_data['cvss3_base_severity'] = cvss3.get('baseSeverity', '')
            cve_data['attack_vector'] = cvss3.get('attackVector', '')
            cve_data['attack_complexity'] = cvss3.get('attackComplexity', '')
            cve_data['privileges_required'] = cvss3.get('privilegesRequired', '')
            cve_data['user_interaction'] = cvss3.get('userInteraction', '')
            cve_data['scope'] = cvss3.get('scope', '')
            cve_data['confidentiality_impact'] = cvss3.get('confidentialityImpact', '')
            cve_data['integrity_impact'] = cvss3.get('integrityImpact', '')
            cve_data['availability_impact'] = cvss3.get('availabilityImpact', '')
        except (KeyError, TypeError):
            cve_data.update({
                'cvss3_vector': '',
                'cvss3_base_score': None,
                'cvss3_base_severity': '',
                'attack_vector': '',
                'attack_complexity': '',
                'privileges_required': '',
                'user_interaction': '',
                'scope': '',
                'confidentiality_impact': '',
                'integrity_impact': '',
                'availability_impact': ''
            })

        # Process CPE matches
        try:
            nodes = cve_item['configurations']['nodes']
            for node in nodes:
                if 'cpe_match' in node:
                    for cpe in node['cpe_match']:
                        cpe_info = {
                            'cve_id': cve_id,
                            'cpe23Uri': cpe.get('cpe23Uri', ''),
                            'vulnerable': cpe.get('vulnerable', False),
                            'versionStartIncluding': cpe.get('versionStartIncluding', ''),
                            'versionEndIncluding': cpe.get('versionEndIncluding', ''),
                            'versionStartExcluding': cpe.get('versionStartExcluding', ''),
                            'versionEndExcluding': cpe.get('versionEndExcluding', '')
                        }

                        # Parse CPE URI into components
                        cpe_parts = cpe_info['cpe23Uri'].split(':')
                        if len(cpe_parts) > 4:
                            cpe_info.update({
                                'vendor': cpe_parts[3],
                                'product': cpe_parts[4],
                                'version': cpe_parts[5]
                            })

                        cpe_items.append(cpe_info)
        except (KeyError, TypeError):
            pass

        cve_items.append(cve_data)

    # Create DataFrames
    cve_df = pd.DataFrame(cve_items)
    cpe_df = pd.DataFrame(cpe_items)

    # Convert date columns to datetime
    date_columns = ['published_date', 'last_modified_date']
    for col in date_columns:
        cve_df[col] = pd.to_datetime(cve_df[col])

    # Sort DataFrames
    cve_df = cve_df.sort_values('id')
    cpe_df = cpe_df.sort_values(['cve_id', 'cpe23Uri'])
    return cve_df, cpe_df


# Replace with your file name and path
file_path = '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2022.json'

try:
    # Process the NVD JSON file
    cve_df, cpe_df = process_nvd_json(file_path)

    # Optionally save to CSV
    cve_df.to_csv('2022_processed_cve_data.csv', index=False)
    cpe_df.to_csv('2022_processed_cpe_data.csv', index=False)

except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except json.JSONDecodeError:
    print("Error: Invalid JSON file format.")
except Exception as e:
    print(f"Error processing file: {str(e)}")

Error: File '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2022.json' not found.


In [8]:
# Helper code used to import NVD data from 2023.
def process_nvd_json(file_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Read the JSON file
    with open(file_path, 'r', encoding = 'ISO-8859-1') as f:
        nvd_data = json.load(f)

    # Lists to store processed items
    cve_items = []
    cpe_items = []

    for cve_item in nvd_data['CVE_Items']:
        cve_data = {}

        # Basic CVE information
        cve_id = cve_item['cve']['CVE_data_meta']['ID']
        cve_data['id'] = cve_id
        cve_data['assigner'] = cve_item['cve']['CVE_data_meta']['ASSIGNER']
        cve_data['published_date'] = cve_item['publishedDate']
        cve_data['last_modified_date'] = cve_item['lastModifiedDate']

        # Description
        descriptions = cve_item['cve']['description']['description_data']
        cve_data['description'] = next((desc['value'] for desc in descriptions if desc['lang'] == 'en'), '')

        # Problem type (CWE)
        try:
            problemtype_data = cve_item['cve']['problemtype']['problemtype_data']
            if problemtype_data and problemtype_data[0]['description']:
                cve_data['cwe'] = problemtype_data[0]['description'][0].get('value', '')
            else:
                cve_data['cwe'] = ''
        except (KeyError, IndexError):
            cve_data['cwe'] = ''

        # References
        try:
            references = cve_item['cve']['references']['reference_data']
            cve_data['references'] = '; '.join(ref['url'] for ref in references)
        except (KeyError, IndexError):
            cve_data['references'] = ''

        # CVSS v3 metrics
        try:
            cvss3 = cve_item['impact']['baseMetricV3']['cvssV3']
            cve_data['cvss3_vector'] = cvss3.get('vectorString', '')
            cve_data['cvss3_base_score'] = cvss3.get('baseScore', None)
            cve_data['cvss3_base_severity'] = cvss3.get('baseSeverity', '')
            cve_data['attack_vector'] = cvss3.get('attackVector', '')
            cve_data['attack_complexity'] = cvss3.get('attackComplexity', '')
            cve_data['privileges_required'] = cvss3.get('privilegesRequired', '')
            cve_data['user_interaction'] = cvss3.get('userInteraction', '')
            cve_data['scope'] = cvss3.get('scope', '')
            cve_data['confidentiality_impact'] = cvss3.get('confidentialityImpact', '')
            cve_data['integrity_impact'] = cvss3.get('integrityImpact', '')
            cve_data['availability_impact'] = cvss3.get('availabilityImpact', '')
        except (KeyError, TypeError):
            cve_data.update({
                'cvss3_vector': '',
                'cvss3_base_score': None,
                'cvss3_base_severity': '',
                'attack_vector': '',
                'attack_complexity': '',
                'privileges_required': '',
                'user_interaction': '',
                'scope': '',
                'confidentiality_impact': '',
                'integrity_impact': '',
                'availability_impact': ''
            })

        # Process CPE matches
        try:
            nodes = cve_item['configurations']['nodes']
            for node in nodes:
                if 'cpe_match' in node:
                    for cpe in node['cpe_match']:
                        cpe_info = {
                            'cve_id': cve_id,
                            'cpe23Uri': cpe.get('cpe23Uri', ''),
                            'vulnerable': cpe.get('vulnerable', False),
                            'versionStartIncluding': cpe.get('versionStartIncluding', ''),
                            'versionEndIncluding': cpe.get('versionEndIncluding', ''),
                            'versionStartExcluding': cpe.get('versionStartExcluding', ''),
                            'versionEndExcluding': cpe.get('versionEndExcluding', '')
                        }

                        # Parse CPE URI into components
                        cpe_parts = cpe_info['cpe23Uri'].split(':')
                        if len(cpe_parts) > 4:
                            cpe_info.update({
                                'vendor': cpe_parts[3],
                                'product': cpe_parts[4],
                                'version': cpe_parts[5]
                            })

                        cpe_items.append(cpe_info)
        except (KeyError, TypeError):
            pass

        cve_items.append(cve_data)

    # Create DataFrames
    cve_df = pd.DataFrame(cve_items)
    cpe_df = pd.DataFrame(cpe_items)

    # Convert date columns to datetime
    date_columns = ['published_date', 'last_modified_date']
    for col in date_columns:
        cve_df[col] = pd.to_datetime(cve_df[col])

    # Sort DataFrames
    cve_df = cve_df.sort_values('id')
    cpe_df = cpe_df.sort_values(['cve_id', 'cpe23Uri'])
    return cve_df, cpe_df


# Replace with your file name and path
file_path = '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2023.json'

try:
    # Process the NVD JSON file
    cve_df, cpe_df = process_nvd_json(file_path)

    # Optionally save to CSV
    cve_df.to_csv('2023_processed_cve_data.csv', index=False)
    cpe_df.to_csv('2023_processed_cpe_data.csv', index=False)

except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except json.JSONDecodeError:
    print("Error: Invalid JSON file format.")
except Exception as e:
    print(f"Error processing file: {str(e)}")

Error: File '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2023.json' not found.


In [9]:
# Helper code used to import NVD data from 2024.
def process_nvd_json(file_path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Read the JSON file
    with open(file_path, 'r', encoding = 'ISO-8859-1') as f:
        nvd_data = json.load(f)

    # Lists to store processed items
    cve_items = []
    cpe_items = []

    for cve_item in nvd_data['CVE_Items']:
        cve_data = {}

        # Basic CVE information
        cve_id = cve_item['cve']['CVE_data_meta']['ID']
        cve_data['id'] = cve_id
        cve_data['assigner'] = cve_item['cve']['CVE_data_meta']['ASSIGNER']
        cve_data['published_date'] = cve_item['publishedDate']
        cve_data['last_modified_date'] = cve_item['lastModifiedDate']

        # Description
        descriptions = cve_item['cve']['description']['description_data']
        cve_data['description'] = next((desc['value'] for desc in descriptions if desc['lang'] == 'en'), '')

        # Problem type (CWE)
        try:
            problemtype_data = cve_item['cve']['problemtype']['problemtype_data']
            if problemtype_data and problemtype_data[0]['description']:
                cve_data['cwe'] = problemtype_data[0]['description'][0].get('value', '')
            else:
                cve_data['cwe'] = ''
        except (KeyError, IndexError):
            cve_data['cwe'] = ''

        # References
        try:
            references = cve_item['cve']['references']['reference_data']
            cve_data['references'] = '; '.join(ref['url'] for ref in references)
        except (KeyError, IndexError):
            cve_data['references'] = ''

        # CVSS v3 metrics
        try:
            cvss3 = cve_item['impact']['baseMetricV3']['cvssV3']
            cve_data['cvss3_vector'] = cvss3.get('vectorString', '')
            cve_data['cvss3_base_score'] = cvss3.get('baseScore', None)
            cve_data['cvss3_base_severity'] = cvss3.get('baseSeverity', '')
            cve_data['attack_vector'] = cvss3.get('attackVector', '')
            cve_data['attack_complexity'] = cvss3.get('attackComplexity', '')
            cve_data['privileges_required'] = cvss3.get('privilegesRequired', '')
            cve_data['user_interaction'] = cvss3.get('userInteraction', '')
            cve_data['scope'] = cvss3.get('scope', '')
            cve_data['confidentiality_impact'] = cvss3.get('confidentialityImpact', '')
            cve_data['integrity_impact'] = cvss3.get('integrityImpact', '')
            cve_data['availability_impact'] = cvss3.get('availabilityImpact', '')
        except (KeyError, TypeError):
            cve_data.update({
                'cvss3_vector': '',
                'cvss3_base_score': None,
                'cvss3_base_severity': '',
                'attack_vector': '',
                'attack_complexity': '',
                'privileges_required': '',
                'user_interaction': '',
                'scope': '',
                'confidentiality_impact': '',
                'integrity_impact': '',
                'availability_impact': ''
            })

        # Process CPE matches
        try:
            nodes = cve_item['configurations']['nodes']
            for node in nodes:
                if 'cpe_match' in node:
                    for cpe in node['cpe_match']:
                        cpe_info = {
                            'cve_id': cve_id,
                            'cpe23Uri': cpe.get('cpe23Uri', ''),
                            'vulnerable': cpe.get('vulnerable', False),
                            'versionStartIncluding': cpe.get('versionStartIncluding', ''),
                            'versionEndIncluding': cpe.get('versionEndIncluding', ''),
                            'versionStartExcluding': cpe.get('versionStartExcluding', ''),
                            'versionEndExcluding': cpe.get('versionEndExcluding', '')
                        }

                        # Parse CPE URI into components
                        cpe_parts = cpe_info['cpe23Uri'].split(':')
                        if len(cpe_parts) > 4:
                            cpe_info.update({
                                'vendor': cpe_parts[3],
                                'product': cpe_parts[4],
                                'version': cpe_parts[5]
                            })

                        cpe_items.append(cpe_info)
        except (KeyError, TypeError):
            pass

        cve_items.append(cve_data)

    # Create DataFrames
    cve_df = pd.DataFrame(cve_items)
    cpe_df = pd.DataFrame(cpe_items)

    # Convert date columns to datetime
    date_columns = ['published_date', 'last_modified_date']
    for col in date_columns:
        cve_df[col] = pd.to_datetime(cve_df[col])

    # Sort DataFrames
    cve_df = cve_df.sort_values('id')
    cpe_df = cpe_df.sort_values(['cve_id', 'cpe23Uri'])
    return cve_df, cpe_df


# Replace with your file name and path
file_path = '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2024.json'

try:
    # Process the NVD JSON file
    cve_df, cpe_df = process_nvd_json(file_path)

    # Optionally save to CSV
    cve_df.to_csv('2024_processed_cve_data.csv', index=False)
    cpe_df.to_csv('2024_processed_cpe_data.csv', index=False)

except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
except json.JSONDecodeError:
    print("Error: Invalid JSON file format.")
except Exception as e:
    print(f"Error processing file: {str(e)}")

Error: File '/content/drive/MyDrive/BIT3474-Group5/nvdcve-1.1-2024.json' not found.


Imports KEV Data.

In [11]:
kev_df = pd.read_csv('/content/drive/MyDrive/BIT3474-Group5/known_exploited_vulnerabilities.csv')

kev_df = kev_df[['cveID', 'vendorProject', 'product', 'vulnerabilityName', 'dateAdded', 'shortDescription', 'requiredAction', 'dueDate', 'knownRansomwareCampaignUse', 'notes', 'cwes']]

kev_df.rename(columns={
    'cveID': 'CVE ID',
    'vendorProject': 'Vendor',
    'product': 'Product',
    'vulnerabilityName': 'Vulnerability Name',
    'dateAdded' : 'Date Added',
    'shortDescription': 'Short Description',
    'requiredAction': 'Required Action',
    'dueDate' : 'Due Date',
    'knownRansomwareCampaignUse': 'Known Ransomware Campaign Use',
    'notes': 'Notes',
    'cwes': 'CWE'
}, inplace=True)

kev_df['Date Added'] = pd.to_datetime(kev_df['Date Added'], errors='coerce')

kev_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BIT3474-Group5/known_exploited_vulnerabilities.csv'

Imports CWE Data.

In [ ]:
# Given Flatten function from Helper Code
def comprehensive_field_extractor(data):
    """
    Extract all fields from CWE weakness data, including nested fields stored as separate columns.

    :param data: Raw CWE weakness data
    :return: Flattened dictionary of all extractable fields
    """
    flattened = {}

    # Core weakness fields
    core_fields = [
        'ID', 'Name', 'Abstraction', 'Structure',
        'Status', 'Description', 'ExtendedDescription', 'LikelihoodOfExploit', 'BackgroundDetails'
    ]
    for field in core_fields:
        flattened[field] = data.get(field, 'N/A')

    # Complex nested fields
    # ApplicablePlatforms
    flattened['Platforms'] = ', '.join([
        f"{p.get('Type', '')}: {p.get('Class', '')}"
        for p in data.get('ApplicablePlatforms', [])
    ])

    # CommonConsequences (split into separate columns)
    common_consequences = data.get('CommonConsequences', [])
    for i, consequence in enumerate(common_consequences):
        flattened[f'Consequence_{i+1}_Scope'] = consequence.get('Scope', 'N/A')
        flattened[f'Consequence_{i+1}_Impact'] = consequence.get('Impact', 'N/A')

    # ObservedExamples (split into separate columns)
    observed_examples = data.get('ObservedExamples', [])
    for i, example in enumerate(observed_examples):
        flattened[f'ObservedExample_{i+1}_Reference'] = example.get('Reference', 'N/A')
        flattened[f'ObservedExample_{i+1}_Description'] = example.get('Description', 'N/A')

    # References (split into separate columns)
    references = data.get('References', [])
    for i, reference in enumerate(references):
        flattened[f'Reference_{i+1}_Title'] = reference.get('Title', 'N/A')
        flattened[f'Reference_{i+1}_URL'] = reference.get('URL', 'N/A')  # Add more fields if needed

    return flattened

In [ ]:
url = 'https://cwe-api.mitre.org/api/v1/cwe/weakness/all'
response = requests.get(url, timeout=60)
data = response.json()

flattened_weaknesses = [
    comprehensive_field_extractor(weakness)
    for weakness in data.get("Weaknesses", [])
]

cwe_df = pd.DataFrame(flattened_weaknesses)

cwe_df.to_csv("cwe_data.csv", index=False)

cwe_df.head()

# **Data Merging**

creates all_cve_data dataframe

In [ ]:
# combines all CVE data into one dataframe. Converts published and modified dates into datetime objects and CVSS3 base scores into numeric objects. The id column is renamed to CVEID for consistency.
# Load data
cve_2022 = pd.read_csv('2022_processed_cve_data.csv')
cve_2023 = pd.read_csv('2023_processed_cve_data.csv')
cve_2024 = pd.read_csv('2024_processed_cve_data.csv')

# Combine into one DataFrame
all_cve_data = pd.concat([cve_2022, cve_2023, cve_2024], ignore_index=True)

all_cve_data['published_date'] = pd.to_datetime(all_cve_data['published_date'], errors='coerce')
all_cve_data['last_modified_date'] = pd.to_datetime(all_cve_data['last_modified_date'], errors='coerce')

all_cve_data['cvss3_base_score'] = pd.to_numeric(all_cve_data['cvss3_base_score'], errors='coerce')

all_cve_data.rename(columns={
    'id': 'CVE ID'
    }, inplace=True)

all_cve_data.to_csv('all_cve_data.csv', index=False)

all_cve_data.head()

creates all_cpe_data dataframe

In [ ]:
# combines all CPE data into one dataframe. Changes cve_id to CVE ID for consistency.
# NaN objects are because not all metadata is required when publishing to the NVD.
cpe_2022 = pd.read_csv('2022_processed_cpe_data.csv')
cpe_2023 = pd.read_csv('2023_processed_cpe_data.csv')
cpe_2024 = pd.read_csv('2024_processed_cpe_data.csv')

all_cpe_data = pd.concat([cpe_2022, cpe_2023, cpe_2024], ignore_index=True)

all_cpe_data.rename(columns={
    'cve_id': 'CVE ID'
    }, inplace=True)

all_cpe_data.to_csv('all_cpe_data.csv', index=False)

all_cpe_data.head()

combines KEV and CWE only keeping the cveID, VulnerabilityName, cwes, Name, and Abstraction columns as per the merged data helper document.

In [ ]:
def extract_cwe_id(cwe_string):
    if pd.isna(cwe_string):
        return None
    match = re.search(r'CWE-(\d+)', cwe_string)
    if match:
        return int(match.group(1))
    return None

kev_df['CWE_ID_Numeric'] = kev_df['CWE'].apply(extract_cwe_id)

kev_df = kev_df.dropna(subset=['CWE_ID_Numeric'])

kev_df['CWE_ID_Numeric'] = kev_df['CWE_ID_Numeric'].astype(int)

cwe_df = pd.read_csv('cwe_data.csv')

cwe_df['ID'] = pd.to_numeric(cwe_df['ID'], errors='coerce')
cwe_df = cwe_df.dropna(subset=['ID'])
cwe_df['ID'] = cwe_df['ID'].astype(int)

combined_df = pd.merge(kev_df, cwe_df, how='left', left_on='CWE_ID_Numeric', right_on='ID')

combined_df = combined_df[['CVE ID', 'Vulnerability Name', 'CWE', 'Name', 'Abstraction']]

combined_df.to_csv('kev_cwe_combined.csv', index=False)

combined_df.head()

Creates the full_df which includes the NVD CVE data, KEV, and CWE. Only displays the columns necessary from the merge data helper.

In [ ]:
full_df = pd.merge(combined_df, all_cve_data, how='left', left_on='CVE ID', right_on='CVE ID')

# Step 7: Select the desired columns
final_df = full_df[['CVE ID', 'Vulnerability Name', 'CWE', 'Name', 'Abstraction', 'assigner', 'published_date']]

# Step 8: Save or view
final_df.to_csv('final_combined_cve_data.csv', index=False)
final_df.head()

prints the final cpe_data df for reference.

In [ ]:
all_cpe_data = pd.read_csv('all_cpe_data.csv')

final_cpe = all_cpe_data[['CVE ID', 'cpe23Uri', 'vulnerable']]

final_cpe.head()

# **Visualizations**

**1. Is there a correlation between the severity of a vulnerability's attack CVSS and its inclusion in the KEV catalog?**

In [ ]:
scored_df = full_df[full_df['cvss3_base_score'].notna()].copy()
scored_df['in_kev'] = scored_df['CVE ID'].isin(kev_df['CVE ID']).astype(str)

plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
sns.boxplot(data=scored_df, x='in_kev', y='cvss3_base_score')
plt.title('CVSS Scores by KEV Inclusion')
plt.xlabel('KEV Inclusion')
plt.ylabel('CVSS3 Base Score')
plt.xticks([0, 1], ['Not Included', 'Included'])
plt.ylim(1, 12)

plt.subplot(1, 2, 2)
sns.violinplot(data=scored_df, x='in_kev', y='cvss3_base_score')
plt.title('CVSS Scores by KEV Inclusion')
plt.xlabel('KEV Inclusion')
plt.ylabel('CVSS3 Base Score')
plt.xticks([0, 1], ['Not Included', 'Included'])
plt.ylim(1, 12)

plt.show()

**2. What is the distribution of vulnerabilities across severity levels for the top vendors in the NVD?**

In [ ]:
vendor_name_mapping = {
    'cve@mitre.org': 'MITRE',
    'audit@patchstack.com': 'Patchstack',
    'cve-request@wordfence.com': 'Wordfence',
    'security-advisories@github.com': 'GitHub',
    'cna@vuldb.com': 'VulDB'
}

vulnerability_count = all_cve_data.groupby('assigner').size().sort_values(ascending=False)

top_5_vendors = vulnerability_count.head(5).index

top_5_data = all_cve_data[all_cve_data['assigner'].isin(top_5_vendors)]

severity_by_vendor = top_5_data.groupby(['cvss3_base_severity', 'assigner']).size().unstack(fill_value=0)

severity_order = ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']
severity_by_vendor = severity_by_vendor.loc[severity_order]

plt.figure(figsize=(12,6))

ax = severity_by_vendor.plot(kind='bar', stacked=True, colormap='tab10')

plt.title('Number of Vulnerabilities for the Top 5 Vendors by Volume and Severity Level')
plt.xlabel('Severity Level')
plt.ylabel('Number of Vulnerabilities')
plt.xticks(rotation=0)

new_labels = [vendor_name_mapping.get(vendor, vendor) for vendor in severity_by_vendor.columns]
plt.legend(title='Vendor', labels=new_labels)

plt.show()

**3. What are the most commonly exploited CWEs across the known exploited CVEs?**

In [ ]:
kev_df = full_df[full_df['published_date'].notna()]

cwe_counts = kev_df['CWE'].value_counts().reset_index()
cwe_counts.columns = ['CWE', 'Count']
cwe_counts = cwe_counts[cwe_counts['CWE'].str.contains("noinfo") == False]

plt.figure(figsize=(12,6))
colors = sns.color_palette('tab10', n_colors=15)
sns.barplot(data=cwe_counts.head(15), y='CWE', x='Count', palette=colors)
plt.title('Most Commonly Exploited Vulnerability CWEs')
plt.xlabel('Number of Exploited CVEs')
plt.ylabel('CWE')

plt.show()

**4. How has the distribution of vulnerability severity levels in the KEV catalog changed over time?**

In [ ]:
kev_df = kev_df.copy()
kev_df['published_date'] = pd.to_datetime(kev_df['published_date'])
kev_df['year_added'] = kev_df['published_date'].dt.year
kev_df = kev_df[kev_df['year_added'] <= 2024]

bins = [0, 3.9, 6.9, 8.9, 10]
labels = ['Low', 'Medium', 'High', 'Critical']
kev_df['severity'] = pd.cut(kev_df['cvss3_base_score'], bins=bins, labels=labels)

severity_by_year = kev_df.groupby(['year_added', 'severity'], observed=True).size().unstack(fill_value=0)

severity_by_year.plot(kind='bar', stacked=True, colormap='tab10', figsize=(12, 6))
plt.title('Vulnerability Severity Distribution by Year')
plt.xlabel('Year')
plt.xticks(rotation=0)
plt.ylabel('Count')
plt.legend(title='Severity')
plt.show()

**5. How do CWE categories relate to average CVSS scores of exploited vulnerabilities?**

In [ ]:
df = kev_df[['CWE', 'cvss3_base_score']].dropna()
df['CWE'] = df['CWE'].str.extract(r'(CWE-\d+)')
avg_scores = df.groupby('CWE')['cvss3_base_score'].mean().reset_index()
pivot = avg_scores.pivot_table(index='CWE', values='cvss3_base_score')
plt.figure(figsize=(10, 8))
sns.heatmap(pivot, cmap='coolwarm')
plt.title('Average CVSS Score by CWE')
plt.xlabel('CVSS Score')
plt.yticks(rotation=0)
plt.ylabel('CWE')
plt.show()

# **Descriptive Statistics**

NVD

In [ ]:
# Total CVEs in the NVD
print('Number of CVEs published in the NVD: \n',len(full_df))

# Range of published dates in the CVE
print('\nTimestamp range: \n', (full_df['published_date'].min(), full_df['published_date'].max()))

# Summary of cvss3 base score including mean, min, max, and the quartiles.
print('\nSummary info: \n', full_df['cvss3_base_score'].describe())

KEV

In [ ]:
# Total number of KEVs
print('Total number of KEVs: \n', len(kev_df))

# Percentage of CVEs that are exploited
kev_count = len(kev_df)
cve_count = len(cve_df)
print('\n Percentage of exploited CVEs: \n', round((kev_count/cve_count) * 100, 2))

# Companies most commonly entered in the KEV
print('\n Most commonly entered companies in KEV: \n', kev_df['assigner'].value_counts().head(10))

# Average CVSS score
print('\n Average CVSS score: \n', kev_df['cvss3_base_score'].mean())

CWE

In [ ]:
# Number of unique CWEs
print('Number of unique CWEs: \n', full_df['CWE'].nunique())

# Most common CWE
print('\nMost common CWE: \n', full_df['CWE'].value_counts().head(1))

# Number of CWE in KEV
print('\nNumber of CWEs in KEV: \n', kev_df['CWE'].nunique())